# Importing Libraries

The libraries that are necessary to provide the dataframe for labelling the tweets:

In [3]:
import pandas as pd
import numpy as np
import json
import re

# Preparation

the general idea of providing such a list is to search through a certain list of keywords into the original data set from Twitter to be able to define which tweet is related to which political party. for example If a tweet has the hashtag for **Justin Trudeau** or liberal or any other liberal parliament members, then that tweet would most likely is talking about something related to liberal post party.

based on the mentioned assumption we use the following link which is the official website for the parliamentarian information of Canada from the link below:

https://lop.parl.ca/sites/ParlInfo/default/en_CA/People/parliamentarians

and download the list of all politician from all the parties since the beginning of the parliament of Canada establishment. the assumption is if any of the tweets for our data sets contain any name or any combination of any of the politicians any of the parties then that tweet is most likely to be related or talking about that political party.Thus, the label for that row of data we'll be assigned to that party for the next steps.

As mentioned before, the main purpose of this project is to work only with the top three political parties in Canada which are Liberals, Conservatives, and NDP. as a result we only kept the rows for the mentioned parties make different combinations of their names, surnames, full name, and some other widely used regular words including "liberal", "conservative", "ndp" in our final dataset for labeling.

In [5]:
# Importing raw dataset
politician_info = pd.read_csv('Data/Parliamentarians.csv')

In [6]:
# Limit the dataset to name and political affiliation
parties_keywords = politician_info[['Name','Political Affiliation']]
parties_keywords['Name'] = parties_keywords['Name'].str.lower()
parties_keywords['Political Affiliation'] = parties_keywords['Political Affiliation'].str.lower()
parties_keywords.dropna()

# Creating new feature name party and rename the palitical affiliation from the original dataset
parties_keywords['party'] = 0
parties_keywords['party'][parties_keywords['Political Affiliation'].str.contains('democratic')] = 'ndp'
parties_keywords['party'][parties_keywords['Political Affiliation'].str.contains('conservative')] = 'conservative'
parties_keywords['party'][parties_keywords['Political Affiliation'].str.contains('liberal')] = 'liberal'

parties_keywords = parties_keywords[parties_keywords['party'] != 0]
parties_keywords = parties_keywords[['Name','party']]


# Adding most common and related words and to the dataset
hashtags = {'Name' : ['cpc','lpc','ndp','liberal','liberals','conservative','conservatives','trudeau','harper','singh'],
            'party' : ['conservative','liberal','ndp','liberal','liberal','conservative','conservative','liberal','conservarive','ndp']}
hashtags = pd.DataFrame(hashtags)
parties_keywords = parties_keywords.append(hashtags)
parties_keywords.reset_index(inplace=True)
parties_keywords.drop('index',axis = 1 ,inplace = True)


# Make other combinations of parliament members name and surname
# Including NameSurname, Surname, Name Surname
test = parties_keywords
test['FName'] = test['Name'].str.split(', ',n = 1, expand = True)[1]
test['FName'] = test['FName'].str.split(' ', expand = True)[0]
test['LName'] = test['Name'].str.split(', ',n = 1, expand = True)[0]
test['LName'] = test['LName'].str.split(' ', expand = True)[0]
test['hashName'] = test['FName'] + test['LName']
test['FullName'] = test['FName'] + ' ' + test['LName']

#keeping only the full name column and the political party
parties_final = test[['Name', 'party']]
hashName_temp = test[['hashName', 'party']]
hashName_temp = hashName_temp.rename(columns={'hashName': 'Name'})
FullName_temp = test[['FullName', 'party']]
FullName_temp = FullName_temp.rename(columns={'FullName': 'Name'})
LName_temp = test[['LName', 'party']]
LName_temp = FullName_temp.rename(columns={'LName': 'Name'})

# merging routine words and hashtags to the processed dataset
parties_final = parties_final.append(hashName_temp, ignore_index= True)
parties_final = parties_final.append(FullName_temp, ignore_index= True)
parties_final = parties_final.append(LName_temp, ignore_index= True)


# save the dataset for further analysis
parties_final.dropna(inplace = True)
parties_final.reset_index(drop=True, inplace=True)
parties_final.to_csv('Data/parties_final.csv')

C:\Users\sorou\AppData\Local\Temp\ipykernel_18208\1735001429.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parties_keywords['Name'] = parties_keywords['Name'].str.lower()
C:\Users\sorou\AppData\Local\Temp\ipykernel_18208\1735001429.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parties_keywords['Political Affiliation'] = parties_keywords['Political Affiliation'].str.lower()
C:\Users\sorou\AppData\Local\Temp\ipykernel_18208\1735001429.py:7: SettingWithCopyWarning: 
A value is trying to be set on a